In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/kaggle/input/real-estate-dataset/data.csv")
df = df.fillna(0)
df.head()

# **EDA**

In [ ]:
#bins=len(np.unique(df_train[[col]].T))//2

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(22,18))
fig.subplots_adjust(hspace=0.25)

for ax, col in zip(axes.flatten(), df.columns):
    ax.hist(df[[col]].values, bins=100)
    ax.set(title=str(col).lower() + " distribution")

In [ ]:
s_fig, s_ax = plt.subplots(figsize=(16, 16))

cols = df.columns
sns.heatmap(df[cols].corr(), ax=s_ax, cmap='RdBu_r', annot=True, center=0.0)

# Train linear models

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop(["CRIM"], axis=1))
y_scaled = scaler.fit_transform(df[["CRIM"]])

In [ ]:
clf_linear = linear_model.LinearRegression()
pred_linear = cross_val_predict(clf_linear, X_scaled, y_scaled, cv=5)

scores_linear = cross_val_score(clf_linear, X_scaled, y_scaled, cv=5, scoring="neg_mean_squared_error")
mse_linear = mean_squared_error(y_scaled, pred_linear)
print (f"linear cross_val_scores: {scores_linear}")
print (f"linear mse_score:        {mse_linear:.10}")

In [ ]:
clf_ridge = linear_model.Ridge(alpha = 100)
pred_ridge = cross_val_predict(clf_ridge, X_scaled, y_scaled, cv=5)

scores_ridge = cross_val_score(clf_ridge, X_scaled, y_scaled, cv=5, scoring="neg_mean_squared_error")
mse_ridge = mean_squared_error(y_scaled, pred_ridge)
print (f"ridge cross_val_scores: {scores_ridge}")
print (f"ridge mse_score:        {mse_ridge:.10}")

In [ ]:
clf_sgd = linear_model.SGDRegressor()
pred_sgd = cross_val_predict(clf_sgd, X_scaled, y_scaled, cv=5)

scores_sgd = cross_val_score(clf_ridge, X_scaled, y_scaled, cv=5, scoring="neg_mean_squared_error")
mse_sgd = mean_squared_error(y_scaled, pred_sgd)
print (f"sgd cross_val_scores: {scores_sgd}")
print (f"sgd mse_score:        {mse_sgd:.10}")

# Submission

In [ ]:
df_output = pd.DataFrame()
df_output["CRIM"] = df["CRIM"]
df_output["pred"] = pred_ridge
df_output["abs_err"] = abs(df_output["CRIM"] - df_output["pred"])

In [ ]:
df_output[["CRIM", "abs_err"]].to_csv("submission_20201114_v2.csv")